# Binary Search Deep Dive

Binary search is the best way of finding a target element in a sorted collection. Here, I will decompose the algorithm into its' most fundamental components, generalize its' functionality, and then apply it to various prototypical leetcode problems.

In [3]:
def binarySearch(sorted, target):
    # returns the index of a specified target element in a sorted array in O(log(n)) time. if sorted does not contain the target, return -1.

    L, R = 0, len(sorted) - 1
    while L <= R:
        mid = (L + R) // 2

        if sorted[mid] > target:
            R = mid - 1
        elif sorted[mid] < target:
            L = mid + 1
        else:
            return mid
    return -1

sorted = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
targetIdx = binarySearch(sorted, 34)
targetIdx

9

## High Level Overview
The key idea here is that when we guess an index of where the target element might be in the array, even if we get it wrong, we still get information regarding where the target element is relative to the guess. Let's say we always guess the middle of the array first. If the target is less than the middle element, we know that the target must be in the bottom half of the array. If the target is greater, it has to be in the upper half. Conversely, consider an unsorted collection. If we get the guess wrong, we don't get information about where to search next. This is why binary search only works on *sorted* arrays.

- We initalize two pointers, a left and a right at the beginning and end elements of the array respectively. This bounds our search space. In order to achieve O(log(n)) runtime, each iteration of our guess needs to divide our search space by a constant factor.

- We start our while loop, and iterate only if the left pointer is less than or equal to the right pointer. This is because if the pointers ever cross each other, that means we've searched the entire array without finding the target, so we exit the loop and return -1. 

- Loop Invariant:
    1) Get the middle value by floor dividing the sum of the L and R indices, ensuring we don't get a float value. The mid index partitions the search space into a lower and upper portion.
    2) If the element at mid is greater than the target, that means the target element is in the lower portion. Therefore, change the R pointer to mid - 1. 
    3) Else, if the element at mid is less than the target, that means the target element is in the upper portion. Therefore, change the L pointer to mid + 1.
    4) Otherwise, return mid

